# Daily Average Temperature - Multi Series Graph

This following graph shows the daily average temperatures for New York, San Francisco and Austin. Data for this graph was obtained from Mike Bostock's tsv file found here: https://gist.githubusercontent.com/mbostock/3884955/raw/e4a63c6dd9388068444a576182acd466de98b603/data.tsv



In [168]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [169]:
HTML('<script src=""//d3js.org/d3.v3.min.js"></script>')

In [170]:
filename  = 'https://gist.githubusercontent.com/mbostock/3884955/raw/e4a63c6dd9388068444a576182acd466de98b603/data.tsv'
avg_temp  = pd.read_csv(filename,"\t")
avg_temp.head()

,date,New York,San Francisco,Austin
0,20111001,63.4,62.7,72.2
1,20111002,58.0,59.9,67.7
2,20111003,53.3,59.1,69.4
3,20111004,55.7,58.8,68.0
4,20111005,64.2,58.7,72.4


#### Convert this to an array of dicts

In [171]:
avg_temp_array_of_dict = avg_temp.to_dict(orient='records')
avg_temp_array_of_dict[:5]

[{'Austin': 72.200000000000003,
  'New York': 63.399999999999999,
  'San Francisco': 62.700000000000003,
  'date': 20111001.0},
 {'Austin': 67.700000000000003,
  'New York': 58.0,
  'San Francisco': 59.899999999999999,
  'date': 20111002.0},
 {'Austin': 69.400000000000006,
  'New York': 53.299999999999997,
  'San Francisco': 59.100000000000001,
  'date': 20111003.0},
 {'Austin': 68.0,
  'New York': 55.700000000000003,
  'San Francisco': 58.799999999999997,
  'date': 20111004.0},
 {'Austin': 72.400000000000006,
  'New York': 64.200000000000003,
  'San Francisco': 58.700000000000003,
  'date': 20111005.0}]

### Adding in the CSS from Mike Bostock's bl.ocks.org http://bl.ocks.org/mbostock/3884955#data.tsv

In [172]:
css_text = '''
.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.x.axis path {
  display: none;
}

.line {
  fill: none;
  stroke: steelblue;
  stroke-width: 1.5px;
}

'''

In [173]:
js_text_template = Template('''

var margin = {top: 20, right: 80, bottom: 30, left: 50},
    width = 1000 - margin.left - margin.right,
    height = 550 - margin.top - margin.bottom;

var parseDate = d3.time.format("%Y%m%d").parse;

var x = d3.time.scale()
    .range([0, width]);

var y = d3.scale.linear()
    .range([height, 0]);

var color = d3.scale.category10();
x
var xAxis = d3.svg.axis()
    .scale(x)
    .orient("bottom");

var yAxis = d3.svg.axis()
    .scale(y)
    .orient("left");

var line = d3.svg.line()
    .interpolate("basis")
    .x(function(d) { return x(d.date); })
    .y(function(d) { return y(d.temperature); });

var svg = d3.select("#$graphdiv").append("svg")
 .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

var data = $python_data;

  color.domain(d3.keys(data[0]).filter(function(key) { return key !== "date"; }));

  data.forEach(function(d) {
    d.date = parseDate(String(d.date));
  });

  var cities = color.domain().map(function(name) {
    return {
      name: name,
      values: data.map(function(d) {
        return {date: d.date, temperature: +d[name]};
      })
    };
  });

  x.domain(d3.extent(data, function(d) { return d.date; }));

  y.domain([
    d3.min(cities, function(c) { return d3.min(c.values, function(v) { return v.temperature; }); }),
    d3.max(cities, function(c) { return d3.max(c.values, function(v) { return v.temperature; }); })
  ]);

svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis);

  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("transform", "rotate(-90)")
      .attr("y", 6)
      .attr("dy", ".71em")
      .style("text-anchor", "end")
      .text("Temperature (F)");

  var city = svg.selectAll(".city")
      .data(cities)
    .enter().append("g")
      .attr("class", "city");

  city.append("path")
      .attr("class", "line")
      .attr("d", function(d) { return line(d.values); })
      .style("stroke", function(d) { return color(d.name); });

  city.append("text")
      .datum(function(d) { return {name: d.name, value: d.values[d.values.length - 1]}; })
      .attr("transform", function(d) { return "translate(" + x(d.value.date) + "," + y(d.value.temperature) + ")"; })
      .attr("x", 3)
      .attr("dy", ".30em")
      .text(function(d) { return d.name; });

''')

In [174]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')
js_text = js_text_template.substitute({'python_data': json.dumps(avg_temp_array_of_dict),
                                       'graphdiv': 'graph-div'})
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))